# Assignmet 2
## Part 1 – Pandas DataFrames

1. Set your working directory and import the dataset Enaho01A-2023-300.csv using Pandas.

*Configuración del Directorio de Trabajo usando Rutas Relativas con ../

In [12]:
import os
import pandas as pd

# 1. Configurar el directorio de trabajo usando rutas relativas con ../
# Para subir 4 niveles desde tu ubicación actual y luego bajar a assingment_2/data
relative_path = "../../../../assingment_2/data"

# 2. Convertir la ruta relativa a una ruta absoluta
absolute_path = os.path.abspath(relative_path)
print(f"Ruta absoluta calculada: {absolute_path}")

# 3. Verificar si la ruta existe
if not os.path.exists(absolute_path):
    print(f"Error: No se encuentra la ruta: {absolute_path}")
    print("Por favor, verifica la estructura de carpetas.")
else:
    # 4. Establecer el directorio de trabajo
    os.chdir(absolute_path)
    print(f"Directorio de trabajo establecido en: {os.getcwd()}")
    
    # 5. Importar el dataset Enaho01A-2023-300.csv
    try:
        # Intentar primero con ISO-8859-10 como se indica en la nota
        df = pd.read_csv("Enaho01A-2023-300.csv", encoding="ISO-8859-10", low_memory=False)
        print("Dataset importado con encoding ISO-8859-10")
    except UnicodeDecodeError:
        try:
            # Si ISO-8859-10 falla, intentar con UTF-8
            df = pd.read_csv("Enaho01A-2023-300.csv", encoding="UTF-8", low_memory=False)
            print("Dataset importado con encoding UTF-8")
        except Exception as e:
            print(f"Error: No se pudo importar el dataset: {e}")
            df = None

Ruta absoluta calculada: C:\Users\HOME\Documents\GitHub\assingment_2\data
Directorio de trabajo establecido en: C:\Users\HOME\Documents\GitHub\assingment_2\data
Dataset importado con encoding ISO-8859-10


*realizar las operaciones solicitadas

In [13]:
if df is not None:
    # a. Leer y mostrar las primeras 5 filas
    print("\nPrimeras 5 filas del dataset:")
    print(df.head())
    
    # b. Convertir los nombres de columna a una lista e imprimirla
    column_names = df.columns.tolist()
    print("\nNombres de columnas como lista:")
    print(column_names)
    
    # c. Verificar los tipos de datos del DataFrame
    print("\nTipos de datos del DataFrame:")
    print(df.dtypes)
    
    # d. Seleccionar un subconjunto con las variables requeridas
    variables_base = ['CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO']
    
    # Seleccionar 4 variables adicionales de interés educativo
    variables_adicionales = [
        'P300A',  # Idioma o lengua materna
        'P301A',  # Último año o grado de estudios y nivel que aprobó - Nivel
        'P302A',  # En los últimos 12 meses, ¿Recibió programa de alfabetización?
        'P304A'   # ¿Cuál es el grado o año de estudios al que asistió el año pasado? - Nivel
    ]
    
    # Verificar que las variables existan en el dataset
    variables_existentes = [col for col in variables_base + variables_adicionales if col in df.columns]
    
    # Crear el subconjunto
    subsample = df[variables_existentes]
    
    print(f"\nSubconjunto seleccionado con {len(variables_existentes)} variables:")
    print(subsample.head())
    
    # Mostrar información sobre el subconjunto
    print(f"\nDimensiones del subconjunto: {subsample.shape}")
    print("\nTipos de datos del subconjunto:")
    print(subsample.dtypes)
     # 7. Renombrar las variables adicionales para mayor claridad
    nuevos_nombres = {
        'P300A': 'lengua_materna',
        'P301A': 'nivel_educativo',
        'P302A': 'programa_alfabetizacion',
        'P304A': 'nivel_asistencia_actual'
    }
    
    subsample = subsample.rename(columns=nuevos_nombres)
    
    print("\nSubconjunto con nombres de variables renombrados:")
    print(subsample.head())
    



Primeras 5 filas del dataset:
    AŅO  MES  CONGLOME  VIVIENDA  HOGAR  CODPERSO  UBIGEO  DOMINIO  ESTRATO  \
0  2023    1      5030         2     11         1   10201        7        4   
1  2023    1      5030         2     11         2   10201        7        4   
2  2023    1      5030         2     11         3   10201        7        4   
3  2023    1      5030         2     11         4   10201        7        4   
4  2023    1      5030        11     11         1   10201        7        4   

   CODINFOR  ...  I311D$5  I311D$6  I311D$7  I3121C I3122C I315B    FACTOR07  \
0         1  ...                                                  118.374542   
1         2  ...                                                  118.374542   
2         2  ...                                                  118.374542   
3         2  ...                          8                       118.374542   
4         1  ...                                                  118.374542   

    FACTORA07

2. Data Manipulation (Data Cleaning):

In [18]:
if df is not None:
    # 1. Explorar el DataFrame usando funciones de resumen
    print("\n=== EXPLORACIÓN INICIAL DEL DATAFRAME ===\n")
    
    # Información general del DataFrame
    print("1. INFORMACIÓN GENERAL DEL DATAFRAME:")
    print(f"Dimensiones: {df.shape} (filas, columnas)")
    print(f"Número total de datos: {df.size}")
    
    # Resumen estadístico de las variables numéricas
    print("\n2. RESUMEN ESTADÍSTICO DE VARIABLES NUMÉRICAS:")
    print(df.describe())
    
    # Información sobre tipos de datos y valores no nulos
    print("\n3. INFORMACIÓN SOBRE TIPOS DE DATOS Y VALORES NO NULOS:")
    print(df.info())
    
    # 2. Identificar valores missing (faltantes)
    print("\n=== IDENTIFICACIÓN DE VALORES MISSING ===\n")
    
    # Contar valores missing por columna
    missing_values = df.isnull().sum()
    print("Valores missing por columna:")
    print(missing_values[missing_values > 0])  # Mostrar solo columnas con valores missing
    
    # Porcentaje de valores missing por columna
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    print("\nPorcentaje de valores missing por columna:")
    print(missing_percentage[missing_percentage > 0])  # Mostrar solo columnas con valores missing
    
    # 3. Manejo de valores missing
    print("\n=== MANEJO DE VALORES MISSING ===\n")
    
    # Estrategia para manejar valores missing:
    # - Para variables categóricas: Podemos usar una categoría específica para missing values
    # - Para variables numéricas: Podemos imputar con la media, mediana o eliminar
    
    # Primero, verificar si hay valores missing en las variables clave
    key_variables = ['CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO', 'P300A', 'P301A', 'P302A', 'P304A']
    key_missing = df[key_variables].isnull().sum()
    
    print("Valores missing en variables clave:")
    print(key_missing)
    
    # Eliminar filas con valores missing en variables clave (si es necesario)
    # Nota: En datasets de encuestas, a veces es mejor no eliminar filas ya que cada una representa una persona/hogar
    
    # Para este ejemplo, eliminaremos filas donde todas las variables clave tienen valores missing
    # Pero primero, verifiquemos cuántas filas serían eliminadas
    rows_before = len(df)
    
    # Eliminar filas donde las variables de identificación tienen valores missing
    # (Estas no deberían tener valores missing en un dataset bien construido)
    df_clean = df.dropna(subset=['CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO'], how='any')
    
    rows_after = len(df_clean)
    rows_removed = rows_before - rows_after
    
    print(f"\nFilas eliminadas por valores missing en variables de identificación: {rows_removed}")
    print(f"Filas restantes: {rows_after}")
    
    # Para variables categóricas con valores missing, podemos asignar una categoría específica
    # Basado en el diccionario de datos, sabemos que los valores missing están codificados como 99 o 9
    
    # 4. Verificar valores específicos codificados como missing (según el diccionario de datos)
    print("\n=== VALORES CODIFICADOS COMO MISSING ===\n")
    
    # Definir los valores que representan missing según el diccionario
    missing_codes = {
        'P300A': [99],  # Lengua materna
        'P301A': [99],  # Nivel educativo
        'P302A': [9],   # Programa de alfabetización
        'P304A': [9]    # Nivel de asistencia actual
    }
    
    # Contar valores codificados como missing
    for column, codes in missing_codes.items():
        if column in df_clean.columns:
            count = df_clean[column].isin(codes).sum()
            print(f"Valores codificados como missing en {column}: {count}")
            
            # Reemplazar valores codificados con NaN
            df_clean[column] = df_clean[column].replace(codes, float('nan'))
    
    # 5. Verificar el estado después de la limpieza
    print("\n=== ESTADO DESPUÉS DE LA LIMPIEZA ===\n")
    
    # Valores missing después de la limpieza
    missing_after = df_clean.isnull().sum()
    print("Valores missing después de la limpieza:")
    print(missing_after[missing_after > 0])
    
    # Para variables numéricas, podemos imputar valores missing con la mediana
    numeric_columns = df_clean.select_dtypes(include=['int64', 'float64']).columns
    
    for col in numeric_columns:
        if df_clean[col].isnull().sum() > 0:
            median_value = df_clean[col].median()
            df_clean[col].fillna(median_value, inplace=True)
            print(f"Imputados {df_clean[col].isnull().sum()} valores missing en {col} con la mediana: {median_value}")
    
    # Para variables categóricas, podemos imputar con la moda o asignar una categoría "Desconocido"
    categorical_columns = df_clean.select_dtypes(include=['object']).columns
    
    for col in categorical_columns:
        if df_clean[col].isnull().sum() > 0:
            mode_value = df_clean[col].mode()[0]
            df_clean[col].fillna(mode_value, inplace=True)
            print(f"Imputados {df_clean[col].isnull().sum()} valores missing en {col} con la moda: {mode_value}")
    
    # 6. Verificación final
    print("\n=== VERIFICACIÓN FINAL ===\n")
    
    # Comprobar si aún hay valores missing
    remaining_missing = df_clean.isnull().sum().sum()
    print(f"Valores missing restantes en todo el DataFrame: {remaining_missing}")
    
    # Resumen del dataset limpio
    print(f"\nDimensiones del dataset después de la limpieza: {df_clean.shape}")
    
    # Guardar el dataset limpio (opcional)
    df_clean.to_csv("enaho_educacion_clean.csv", index=False, encoding='utf-8')
    print("\nDataset limpio guardado como 'enaho_educacion_clean.csv'")
    
    # Mostrar información del dataset limpio
    print("\nInformación del dataset limpio:")
    print(df_clean.info())
    
else:
    print("No se pudo realizar la limpieza de datos debido a problemas con la importación del dataset.")
       


=== EXPLORACIÓN INICIAL DEL DATAFRAME ===

1. INFORMACIÓN GENERAL DEL DATAFRAME:
Dimensiones: (108354, 511) (filas, columnas)
Número total de datos: 55368894

2. RESUMEN ESTADÍSTICO DE VARIABLES NUMÉRICAS:
            AŅO            MES       CONGLOME       VIVIENDA          HOGAR  \
count  108354.0  108354.000000  108354.000000  108354.000000  108354.000000   
mean     2023.0       6.495127   16944.756797      77.820330      11.146271   
std         0.0       3.445244    3144.386733      68.547022       1.370084   
min      2023.0       1.000000    5007.000000       1.000000      11.000000   
25%      2023.0       3.000000   16028.000000      31.000000      11.000000   
50%      2023.0       7.000000   17500.000000      66.000000      11.000000   
75%      2023.0       9.000000   19014.000000     106.000000      11.000000   
max      2023.0      12.000000   21001.000000     991.000000      44.000000   

            CODPERSO         UBIGEO        DOMINIO        ESTRATO  \
count  10835

 3. Importación y Manipulación del Segundo Dataset
    

In [21]:
# Importar el segundo dataset (módulo de vivienda) con el nombre correcto
try:
    df_vivienda = pd.read_csv("Enaho01-2023-200.csv", encoding="ISO-8859-10", low_memory=False)
    print("Dataset de vivienda importado con encoding ISO-8859-10")
except UnicodeDecodeError:
    try:
        df_vivienda = pd.read_csv("Enaho01-2023-200.csv", encoding="UTF-8", low_memory=False)
        print("Dataset de vivienda importado con encoding UTF-8")
    except Exception as e:
        print(f"Error: No se pudo importar el dataset de vivienda: {e}")
        df_vivienda = None

# Si la importación fue exitosa, realizar las operaciones solicitadas
if df_vivienda is not None:
    # 1. Display the first 5 rows
    print("\nPrimeras 5 filas del dataset de vivienda:")
    print(df_vivienda.head())
    
    # 2. Convert the column names into a list and print it
    column_names_vivienda = df_vivienda.columns.tolist()
    print("\nNombres de columnas del dataset de vivienda:")
    print(column_names_vivienda)
    
    # 3. Check the data types
    print("\nTipos de datos del dataset de vivienda:")
    print(df_vivienda.dtypes)
    
    # 4. Select a subsample with the required variables
    variables_base = ['CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO']
    
    # Seleccionar variables adicionales que SÍ existen en el dataset
    # Basado en las columnas disponibles que vimos en el output
    variables_adicionales_vivienda = [
        'P203',   # ¿Cuántos cuartos o habitaciones tiene su hogar? (excluye baños y cocina)
        'P204',   # ¿En cuántos de estos cuartos duermen las personas del hogar?
        'P205',   # ¿La vivienda que ocupa el hogar es?
        'P206',   # Material de las paredes
        'P207'    # Material de los pisos
    ]
    
    # Verificar que las variables existan en el dataset
    variables_existentes_vivienda = [col for col in variables_base + variables_adicionales_vivienda if col in df_vivienda.columns]
    
    # Crear el subconjunto
    subsample_vivienda = df_vivienda[variables_existentes_vivienda]
    
    print(f"\nSubconjunto seleccionado con {len(variables_existentes_vivienda)} variables:")
    print(subsample_vivienda.head())
    
    # 5. Perform the following modifications:
    print("\n=== MODIFICACIONES AL SUBCONJUNTO ===\n")
    
    # A. Change the data type of a variable
    print("Tipos de datos antes de las modificaciones:")
    print(subsample_vivienda.dtypes)
    
    # Convertir P205 (tipo de vivienda) a categórica
    if 'P205' in subsample_vivienda.columns:
        print(f"\nA. Cambiando tipo de dato de P205 de {subsample_vivienda['P205'].dtype} a category")
        subsample_vivienda['P205'] = subsample_vivienda['P205'].astype('category')
        print(f"Tipo de dato de P205 después del cambio: {subsample_vivienda['P205'].dtype}")
    
    # B. Modify some values in a specific column
    # Modificar valores en la columna P206 (Material de las paredes)
    if 'P206' in subsample_vivienda.columns:
        print("\nB. Modificando valores en la columna P206 (Material de las paredes):")
        print("Valores originales en P206:")
        print(subsample_vivienda['P206'].value_counts())
        
        # Crear un mapeo de valores (basado en el diccionario de la ENAHO)
        # Nota: Estos valores deben verificarse con el diccionario oficial
        mapeo_p206 = {
            "1": "Ladrillo o bloque de cemento",
            "2": "Adobe o tapia",
            "3": "Quincha",
            "4": "Piedra con barro",
            "5": "Piedra con cal o cemento",
            "6": "Madera",
            "7": "Estera",
            "8": "Otro material"
        }
        
        # Aplicar el mapeo
        subsample_vivienda['P206_modificado'] = subsample_vivienda['P206'].map(mapeo_p206)
        
        print("Valores modificados en P206_modificado:")
        print(subsample_vivienda['P206_modificado'].value_counts())
    
    # C. Additional modification: Convertir P203 y P204 a numéricas
    if 'P203' in subsample_vivienda.columns:
        print(f"\nC. Convirtiendo P203 a numérico")
        # Algunos valores podrían ser strings, así que los convertimos a numéricos, forzando los no numéricos a NaN
        subsample_vivienda['P203'] = pd.to_numeric(subsample_vivienda['P203'], errors='coerce')
        print(f"Tipo de dato de P203 después del cambio: {subsample_vivienda['P203'].dtype}")
    
    if 'P204' in subsample_vivienda.columns:
        print(f"\nD. Convirtiendo P204 a numérico")
        subsample_vivienda['P204'] = pd.to_numeric(subsample_vivienda['P204'], errors='coerce')
        print(f"Tipo de dato de P204 después del cambio: {subsample_vivienda['P204'].dtype}")
    
    # Mostrar el resultado final
    print("\nSubconjunto después de las modificaciones:")
    print(subsample_vivienda.head())
    print("\nTipos de datos después de las modificaciones:")
    print(subsample_vivienda.dtypes)
    
    # Guardar el subconjunto modificado
    subsample_vivienda.to_csv("subsample_vivienda_modificado.csv", index=False, encoding='utf-8')
    print("\nSubconjunto modificado guardado como 'subsample_vivienda_modificado.csv'")
    
else:
    print("No se pudo realizar el análisis del dataset de vivienda debido a problemas con la importación.")

Dataset de vivienda importado con encoding ISO-8859-10

Primeras 5 filas del dataset de vivienda:
    AŅO  MES  CONGLOME  VIVIENDA  HOGAR  CODPERSO  UBIGEO  DOMINIO  ESTRATO  \
0  2023    2      5007        22     11         1   10101        4        4   
1  2023    2      5007        22     11         2   10101        4        4   
2  2023    2      5007        22     11         3   10101        4        4   
3  2023    2      5007        31     11         1   10101        4        4   
4  2023    2      5007        31     11         2   10101        4        4   

               P201P  ...  OCUPAC_R3 OCUPAC_R4 RAMA_R3 RAMA_R4 CODTAREA  \
0  20190050070221101  ...                                                 
1  20190050070221102  ...                                                 
2  20190050070221104  ...                                                 
3  20190050070311102  ...                                                 
4  20230050070311102  ...                           

C:\Users\HOME\AppData\Local\Temp\ipykernel_13000\4231310518.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample_vivienda['P205'] = subsample_vivienda['P205'].astype('category')
C:\Users\HOME\AppData\Local\Temp\ipykernel_13000\4231310518.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample_vivienda['P206_modificado'] = subsample_vivienda['P206'].map(mapeo_p206)
C:\Users\HOME\AppData\Local\Temp\ipykernel_13000\4231310518.py:93: SettingWithCopyWarning: 
A value is trying to be set on a cop


Subconjunto modificado guardado como 'subsample_vivienda_modificado.csv'
